In [160]:
import os
import shutil
import json
from pathlib import Path
from dataclasses import dataclass
from configparser import ConfigParser
import yaml
import re
import pandas as pd
#from tqdm.auto import tqdm
from datetime import datetime
import markdown
import nbconvert
from collections import defaultdict
from dataclasses import dataclass

from pymdownx import emoji

from lxml.html.clean import clean_html
import nbformat

from IPython.display import display, Markdown

%load_ext autoreload
%autoreload 2
import blog

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [115]:
def printmd(text):
    display(Markdown(text))

## config

All the config is saved in a config.ini file

In [128]:
config = ConfigParser()
config.read("config.ini")
for section in config.sections():
    print(f"{section}: {config.items(section)}")

blog: [('title', '"This is a blog"'), ('baseurl', '""'), ('default_post_title', '"this is an untitled post"'), ('default_post_type', 'post'), ('header', 'khalido.org'), ('header_text', 'I love me some subheader text.')]
paths: [('posts', 'notes'), ('notebooks', 'notebooks'), ('publish', 'public'), ('static', 'static')]


In [129]:
path_md = blog.path_md
path_nb = blog.path_nb
path_publish = blog.path_publish
path_static = blog.path_static
path_md, path_publish, path_static

(WindowsPath('notes'), WindowsPath('public'), WindowsPath('static'))

In [130]:
posts, postsdict, tags = blog.get_posts()
tags # dict of tags to posts. sort this too?

Converting 3 notebooks to markdown files.
notebooks\Advent-of-Code-2015.ipynb
notebooks\blogging_with_jupyter_notebooks.ipynb
notebooks\monty_hall.ipynb
--------------------------



defaultdict(set,
            {'python': {'Advent-of-Code-2015',
              'advent-of-code-2016',
              'anaconda',
              'blogging_with_jupyter_notebooks',
              'building_this_blog',
              'monty_hall',
              'note_in_subfolder',
              'udacity-cs101'},
             'untagged': {'apache-spark',
              'blogs-i-like',
              'chrome',
              'deeplearning-ai-part-1-of-5',
              'deeplearning-ai-part-2-of-5',
              'deeplearning-ai-part-3-of-5',
              'dss-meetup',
              'fastai-part1-2018',
              'google-ml-best-practice',
              'grokking-deep-learning-book',
              'harvard-cs109-notes',
              'mitx6002x',
              'ms_openhack_2018',
              'my-first-post',
              'setup-mac',
              'setup-ubuntu',
              'sml-meetup',
              'sml-meetup-2018-09',
              'techfugees-sydney-hackathon-2018',
             

In [100]:
for t in tags.keys():
    print(f"tag: {t}")
    for s in sorted(tags[t], key = lambda x: postsdict[x].date, reverse=True):
        print(postsdict[s].slug, postsdict[s].date, postsdict[s].path)
        #print(postsdict[s].html[:22])
    print("---------------")

tag: python
blogging_with_jupyter_notebooks 2020-08-09 tmp\blogging_with_jupyter_notebooks.md
anaconda 2020-07-25 notes\anaconda.md
building_this_blog 2020-07-14 notes\building_this_blog.md
note_in_subfolder 2020-06-14 notes\sub_notes\note_in_subfolder.md
monty_hall 2019-07-05 tmp\monty_hall.md
---------------
tag: books
stats-done-wrong-book 2020-07-25 notes\stats-done-wrong-book.md
---------------
tag: statistics
stats-done-wrong-book 2020-07-25 notes\stats-done-wrong-book.md
---------------
tag: apps
vs_code 2019-10-10 notes\vs_code.md
---------------
tag: recipe
daal 2019-08-04 notes\daal.md
---------------
tag: jupyter
jupyterlab 2019-08-04 notes\jupyterlab.md
---------------
tag: chromebooks
setup-crostini-chromebook 2018-08-24 notes\setup-crostini-chromebook.md
---------------


In [135]:
for p in posts:
    print(p.type, len(p.type))

post 4
post 4
post 4
post 4
post 4
post 4
post 4
post 4
post 4
post 4
post 4
post 4
post 4
post 4
post 4
post 4
post 4
post 4
post 4
post 4
notebook 8
notebook 8
post 4
post 4
post 4
post 4
post 4
post 4
post 4
notebook 8
post 4
post 4
post 4


In [136]:
for tag, pp in tags.items():
    print(tag, pp)

python {'advent-of-code-2016', 'udacity-cs101', 'blogging_with_jupyter_notebooks', 'monty_hall', 'note_in_subfolder', 'Advent-of-Code-2015', 'building_this_blog', 'anaconda'}
untagged {'setup-ubuntu', 'sml-meetup-2018-09', 'setup-mac', 'blogs-i-like', 'sml-meetup', 'my-first-post', 'deeplearning-ai-part-2-of-5', 'udacity-nd101', 'deeplearning-ai-part-1-of-5', 'grokking-deep-learning-book', 'ms_openhack_2018', 'dss-meetup', 'mitx6002x', 'techfugees-sydney-hackathon-2018', 'harvard-cs109-notes', 'apache-spark', 'google-ml-best-practice', 'deeplearning-ai-part-3-of-5', 'fastai-part1-2018', 'chrome'}
books {'stats-done-wrong-book'}
statistics {'stats-done-wrong-book'}
apps {'vs_code'}
recipe {'daal'}
jupyter {'jupyterlab'}
chromebooks {'setup-crostini-chromebook'}
courses {'udacity-cs101'}


In [137]:
for p in posts:
    print(p.date, p.title, p.slug[-10:])

2020-08-20 Advent of Code 2016 -code-2016
2020-08-20 Apache Spark ache-spark
2020-08-20 Data Science blogs ogs-i-like
2020-08-20 Gooogle Chrome chrome
2020-08-20 deeplearning.ai: Neural Networks and Deep Learning art-1-of-5
2020-08-20 deeplearning.ai: Improving Deep Neural Networks art-2-of-5
2020-08-20 deeplearning.ai: Structuring Machine Learning Projects art-3-of-5
2020-08-20 DSS-2018-07: Michael Allwright and Inna Kolyshkina dss-meetup
2020-08-20 fastai Practical Deep Learning For Coders, Part 1, 2018 part1-2018
2020-08-20 Googles Best Practices for ML Engineering t-practice
2020-08-20 Grokking Deep Learning by Andrew Trask rning-book
2020-08-20 Notes for Harvard's Cs109 data science class s109-notes
2020-08-20 MITx 6.00.2X: Intro to Computational Thinking and Data Science mitx6002x
2020-08-20 Microsoft OpenHack 2018 Sydney nhack_2018
2020-08-20 MacOS setup-mac
2020-08-20 Setup Linux tup-ubuntu
2020-08-20 SML 2018-09: Deep Reinforcement Learning up-2018-09
2020-08-20 SML 2018-08: D

In [85]:
for slug, p in postsdict.items():
    print(slug, p.date)
    if p.toc:
        print("has toc")
    else:
        print("no toc")

test 2020-07-28
has toc
anaconda 2020-07-25
no toc
stats-done-wrong-book 2020-07-25
no toc
building_this_blog 2020-07-14
has toc
note_in_subfolder 2020-06-14
has toc
vs_code 2019-10-10
no toc
jupyterlab 2019-08-04
no toc
monty_hall 2019-07-05
no toc
setup-crostini-chromebook 2018-08-24
has toc


## Pinboard

In [11]:
pb_token = os.environ.get("PB_TOKEN")
pb_token[:4]

TypeError: 'NoneType' object is not subscriptable

In [13]:
import pinboard
pb = pinboard.Pinboard(pb_token)

# check its working
pb.posts.update() # should return most recent updated date

datetime.datetime(2020, 7, 10, 22, 49, 29)

In [175]:
all_posts = pb.posts.all() # list of all bookmarks

In [196]:
# the api returns a class obj
bookmark = all_posts[0]
# useful fields in that object:
bookmark.description, bookmark.tags, bookmark.extended, bookmark.url, bookmark.time

('What I learned from looking at 200 machine learning tools',
 ['ai'],
 '',
 'https://huyenchip.com/2020/06/22/mlops.html',
 datetime.datetime(2020, 6, 24, 4, 39, 45))

In [149]:
links = pb.posts.recent()

In [171]:
for b in links["posts"]:
    print(b.time, b.tags)

2020-06-24 04:39:45 ['ai']
2020-06-22 03:38:00 ['python']
2020-06-22 03:36:08 ['python']
2020-06-15 06:40:30 ['apps', 'markdown']
2020-06-14 07:20:02 ['windows', 'apps']
2020-06-13 20:06:16 ['']
2020-06-13 12:18:36 ['recipes']
2020-06-13 05:03:24 ['games']
2020-06-12 14:13:48 ['dataviz']
2020-06-12 01:04:52 ['art', 'posters']
2020-06-10 23:44:52 ['ai']
2020-06-04 05:10:27 ['blogs']
2020-06-04 02:11:58 ['analytics']
2020-06-02 07:29:04 ['streamlit']
2020-05-31 03:00:33 ['books', 'programming']


In [172]:
tag_posts = pb.posts.all(tag=["programming"])

In [174]:
for b in tag_posts:
    print(b.time, b.tags)

2020-05-31 03:00:33 ['books', 'programming']
2018-03-14 07:07:19 ['programming']
2017-02-27 23:27:15 ['python', 'programming', 'bayes']
2016-11-15 22:35:31 ['programming', 'poster']
2016-11-15 11:32:49 ['programming']


## Read each file and parse front matter

In [138]:
from markdown.extensions.codehilite import CodeHilite
codehilite = CodeHilite(noclasses=True)

In [87]:
txt = """

This is a "test" para. Here goes a line break:
what does it do?

```python
print(3+3) # this is a comment
```

## Bash

Most remote VM's are running a basic bash shell. Fix that by running

```bash
sudo apt update &&
sudo apt install htop neofetch -y &&
wget https://github.com/sharkdp/bat/releases/download/v0.15.1/bat-musl_0.15.1_amd64.deb &&
sudo dpkg -i bat-musl_0.15.1_amd64.deb &&
rm bat-musl_0.15.1_amd64.deb &&
sh -c "$(curl -fsSL https://raw.github.com/ohmybash/oh-my-bash/master/tools/install.sh)"
```

## repos

This updates repos - but doesn't upgrade as no need generally for a short lived VM and installs

- [htop](https://hisham.hm/htop/): monitor machine use
- neofetch: show machine overview
- [Bat](https://github.com/sharkdp/bat): better cat with syntax highlighting
- [oh-my-bash](https://ohmybash.github.io/) - makes the terminal look good

If working with files:

```bash
sudo apt install fzf mc -y
```

- mc: file manager
- fzf: fuzzy search

## git

Setup git by running the following inside a git repo (if needed).

```bash
git config --global user.name "khalido" &&
git config --global user.email "khalid.omar@gmail.com" &&
git config credential.helper 'cache --timeout=21600'
```



Note, there has gotta be a better way to do this.
"""

In [140]:
# make enters into line breaks by adding "nl2br"
extensions=["extra", "toc", "codehilite"] # , "smarty" 

# https://help.farbox.com/pygments.html
# monokai default

extension_configs = {
    'codehilite': {
        'noclasses': False,
        'linenums': False,
        'pygments_style': "monokai"
    },
}

In [161]:
print(md.toc)

<div class="toc">
<ul>
<li><a href="#twitter-embed">twitter embed</a></li>
</ul>
</div>



In [157]:
# configure python markdown parser

# make enters into line breaks by adding "nl2br"
extensions=["extra", "toc", "codehilite"] # , "smarty" 

# https://help.farbox.com/pygments.html
# monokai default

extension_configs = {
    "output_format": "html5",
    'codehilite': {
        'noclasses': False,
        'linenums': False,
        'pygments_style': "monokai"
    },
}

md = markdown.Markdown(extensions=extensions, extension_configs=extension_configs)

# paths
md_paths = [f for f in path_md.rglob("*.md")]
notebook_paths = [f for f in path_nb.rglob("*.ipynb")]

data = [] # holds all the posts
tags = defaultdict(set) # holds set of all posts for every tag

for p in md_paths:
    d = {} # one dict for each post
    all_txt = p.read_text() 

    # extract front matter b/w "---" lines
    n = all_txt[3:].find("---") + 3
    fm = yaml.load(all_txt[:n]) # front matter dict
    txt = all_txt[n+3:].strip() # text excluding front matter

    # now add values I care about to the dict
    d["title"] = fm["title"]
    
    if fm.get("slug"):
        d["slug"] = fm["slug"]
    else:
        d["slug"] = p.name.split(".")[0]

    # add created date
    try:
        d["date"] = fm["date"]
    except:
        dt = datetime.fromtimestamp(p.stat().st_ctime) # create time of file
        d["date"] = dt

    # add last modified data
    try:
        d["lastmod"] = fm["lastmod"]
    except:
        dt = datetime.fromtimestamp(p.stat().st_mtime)
        d["lastmod"] = dt
    
    d["path"] = p
    d["filename"] = p.name
    d["front_matter"] = fm
    d["markdown"] = txt
    
    d["tags"] = fm.get("tags", ["untagged"])
    for tag in d["tags"]:
        tags[tag].add(d["slug"])

    d["html"] = md.convert(txt) 
    
    data.append(d)

df = pd.DataFrame(data) #.set_index("filename")
print(f"{len(df)} posts converted and saved in dataframe")
df

30 posts converted and saved in dataframe


,title,slug,date,lastmod,path,filename,front_matter,markdown,tags,html
0,Advent of Code 2016,advent-of-code-2016,2020-08-20 10:42:28.813488,2020-08-20 11:04:43.787251,notes\advent-of-code-2016.md,advent-of-code-2016.md,"{'title': 'Advent of Code 2016', 'description'...",todo: convert this into a jupyter notebook and...,[python],<p>todo: convert this into a jupyter notebook ...
1,Using Anaconda,anaconda,2020-07-25 08:35:00.889113,2020-08-20 10:39:12.808777,notes\anaconda.md,anaconda.md,"{'title': 'Using Anaconda', 'tags': ['python']...",## Install Anaconda\n\nThere seems to be two m...,[python],"<h2 id=""install-anaconda"">Install Anaconda</h2..."
2,Apache Spark,apache-spark,2020-08-20 10:42:28.815490,2020-08-11 16:06:37.295450,notes\apache-spark.md,apache-spark.md,"{'title': 'Apache Spark', 'description': 'star...",Way back in the good old days Google came up w...,[untagged],<p>Way back in the good old days Google came u...
3,Data Science blogs,blogs-i-like,2020-08-20 10:42:28.862486,2020-08-11 16:06:37.305451,notes\blogs-i-like.md,blogs-i-like.md,"{'title': 'Data Science blogs', 'description':...","Blogs on data science, AI, machine learning et...",[untagged],"<p>Blogs on data science, AI, machine learning..."
4,how the blog was built,building_this_blog,2020-07-14 00:00:00.000000,2020-08-20 10:40:52.939441,notes\building_this_blog.md,building_this_blog.md,"{'title': 'how the blog was built', 'date': 20...",Hereby I talk through writing my own blog engi...,[python],<p>Hereby I talk through writing my own blog e...
5,Gooogle Chrome,chrome,2020-08-20 10:42:28.877490,2020-08-11 16:06:37.310451,notes\chrome.md,chrome.md,"{'title': 'Gooogle Chrome', 'description': 'ma...",## Flags\n\nSome of these flags get set by \n\...,[untagged],"<h2 id=""flags"">Flags</h2>\n<p>Some of these fl..."
6,Daal (lentils),daal,2019-08-04 00:00:00.000000,2020-08-04 09:47:15.308486,notes\daal.md,daal.md,"{'title': 'Daal (lentils)', 'date': 2019-08-04...",A simple daal recipe.\n\n1 cup of daal is appr...,[recipe],<p>A simple daal recipe.</p>\n<p>1 cup of daal...
7,deeplearning.ai: Neural Networks and Deep Lear...,deeplearning-ai-part-1-of-5,2020-08-20 10:42:28.810488,2020-08-11 16:06:37.294447,notes\deeplearning-ai-part-1-of-5.md,deeplearning-ai-part-1-of-5.md,{'title': 'deeplearning.ai: Neural Networks an...,It had been a while since I finished Udacity's...,[untagged],<p>It had been a while since I finished Udacit...
8,deeplearning.ai: Improving Deep Neural Networks,deeplearning-ai-part-2-of-5,2020-08-20 10:42:28.830485,2020-08-11 16:06:37.298453,notes\deeplearning-ai-part-2-of-5.md,deeplearning-ai-part-2-of-5.md,{'title': 'deeplearning.ai: Improving Deep Neu...,[Improving Deep Neural Networks: Hyperparamete...,[untagged],"<p><a href=""https://www.coursera.org/learn/dee..."
9,deeplearning.ai: Structuring Machine Learning ...,deeplearning-ai-part-3-of-5,2020-08-20 10:42:28.836485,2020-08-11 16:06:37.299448,notes\deeplearning-ai-part-3-of-5.md,deeplearning-ai-part-3-of-5.md,{'title': 'deeplearning.ai: Structuring Machin...,This course covers how to think about and impr...,[untagged],<p>This course covers how to think about and i...


In [171]:
html = md.convert(txt)
print(html)

<p>This note is inside a subfolder.</p>
<h2 id="twitter-embed">twitter embed</h2>
<p>This should show a twitter tweet embedded inside the page:</p>
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Sunsets don&#39;t get much better than this one over <a href="https://twitter.com/GrandTetonNPS?ref_src=twsrc%5Etfw">@GrandTetonNPS</a>. <a href="https://twitter.com/hashtag/nature?src=hash&amp;ref_src=twsrc%5Etfw">#nature</a> <a href="https://twitter.com/hashtag/sunset?src=hash&amp;ref_src=twsrc%5Etfw">#sunset</a> <a href="http://t.co/YuKy2rcjyU">pic.twitter.com/YuKy2rcjyU</a></p>&mdash; US Department of the Interior (@Interior) <a href="https://twitter.com/Interior/status/463440424141459456?ref_src=twsrc%5Etfw">May 5, 2014</a></blockquote>

<p><script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script></p>
<p>The code used</p>
<div class="codehilite"><pre><span></span><code><span class="p">&lt;</span><span class="nt">blockquote</span> <span class="na">class

In [186]:
from lxml.html.clean import Cleaner

cleaner = Cleaner(page_structure=False, kill_tags=["script", "img"], scripts=True, javascript=True)
print(cleaner.clean_html(txt))

<div><p>This note is inside a subfolder.

## twitter embed

This should show a twitter tweet embedded inside the page:

</p><blockquote class="twitter-tweet"><p lang="en" dir="ltr">Sunsets don't get much better than this one over <a href="https://twitter.com/GrandTetonNPS?ref_src=twsrc%5Etfw">@GrandTetonNPS</a>. <a href="https://twitter.com/hashtag/nature?src=hash&amp;ref_src=twsrc%5Etfw">#nature</a> <a href="https://twitter.com/hashtag/sunset?src=hash&amp;ref_src=twsrc%5Etfw">#sunset</a> <a href="http://t.co/YuKy2rcjyU">pic.twitter.com/YuKy2rcjyU</a></p>— US Department of the Interior (@Interior) <a href="https://twitter.com/Interior/status/463440424141459456?ref_src=twsrc%5Etfw">May 5, 2014</a></blockquote> 

The code used

```html
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Sunsets don't get much better than this one over <a href="https://twitter.com/GrandTetonNPS?ref_src=twsrc%5Etfw">@GrandTetonNPS</a>. <a href="https://twitter.com/hashtag/nature?src=hash&amp;ref_src=t

## Output

In [12]:
posts, postsdict, tags = blog.get_posts()
print(type(posts), type(postsdict), type(tags))
print(tags.keys())

printmd("\n**Post slugs**")
for post in posts:
    print(post.link, post.title)

converting notebooks\monty_hall.ipynb

converted 2 notebooks to markdown files
converting notebooks\.ipynb_checkpoints\monty_hall-checkpoint.ipynb

converted 2 notebooks to markdown files
<class 'list'> <class 'dict'> <class 'collections.defaultdict'>
dict_keys(['python', 'jupyter', 'chromebooks', 'books', 'statistics', 'apps'])



**Post slugs**

python/anaconda.html Using Anaconda
books/stats-done-wrong-book.html Statistics Done Wrong: The Woefully Complete Guide
python/building_this_blog.html how the blog was built
python/note_in_subfolder.html Testing note in a subfolder
apps/vs_code.html Visual Studio Code
jupyter/jupyterlab.html Jupyter Lab tips and tricks
python/monty_hall-checkpoint.html Monty Hall
python/monty_hall.html Monty Hall
chromebooks/setup-crostini-chromebook.html Setting up Linux on a Chromebook with Crostini


In [17]:
from mako.template import Template
from mako.lookup import TemplateLookup

lookup = TemplateLookup(directories=["templates"])

In [18]:
def search_json(posts, path=path_publish):
    """makes a json of the posts info to search, returns json string
    and writes to disk"""

    search_list = [
        {"title": post.title, "tags": post.tags, "link": post.link} 
        for post in posts]

    posts_json = json.dumps(search_list)
    path = path / "posts.json"
    make_folder(path)
    
    path.write_text(posts_json)
    return posts_json

postsjson = search_json(posts)
postsjson

'[{"title": "Using Anaconda", "tags": ["python"], "link": "python/anaconda.html"}, {"title": "Statistics Done Wrong: The Woefully Complete Guide", "tags": ["books", "statistics"], "link": "books/stats-done-wrong-book.html"}, {"title": "how the blog was built", "tags": ["python"], "link": "python/building_this_blog.html"}, {"title": "Testing note in a subfolder", "tags": ["python"], "link": "python/note_in_subfolder.html"}, {"title": "Visual Studio Code", "tags": ["apps"], "link": "apps/vs_code.html"}, {"title": "Jupyter Lab tips and tricks", "tags": ["jupyter"], "link": "jupyter/jupyterlab.html"}, {"title": "Monty Hall", "tags": ["python"], "link": "python/monty_hall.html"}, {"title": "Setting up Linux on a Chromebook with Crostini", "tags": ["chromebooks"], "link": "chromebooks/setup-crostini-chromebook.html"}]'

In [19]:
def make_folder(path, debug: bool=False):
    """makes parent folders if they don't exist for a path"""

    # assume the last thing is a file if there is a dot in the name
    if "." in path.name:
        path = path.parent
    
    try:
        path.mkdir(parents=True, exist_ok=False)
        if debug: print(f"{path} folder made")
    except FileExistsError:
        if debug: print(f"{path} folder already exists")
        pass

make_folder(path=path_publish / "test" / "test12"/ "test.html", debug=True)

dist/test/test12 folder already exists


In [20]:
def write_posts(tmpl:str = "post.html", posts=posts):
    """makes a html index page in the foldername using list of posts passed in
    
    Args:
        tmpl: name of templete
        posts: list of posts
    """
    
    template = lookup.get_template(tmpl)

    for post in posts:
        html = template.render(post=post).strip()
        
        # publish each post inside its first tag folder
        if post.tags:
            path = path_publish / post.tags[0]
        else:
            path = path_publish

        make_folder(path) # make folder in case

        path = path / f"{post.slug}.html"
        
        path.write_text(html)
        print(f"wrote {post.slug} to {path}")


def write_index_page(tmpl:str = "index.html", foldername=None, posts=posts):
    """makes a html index page in the foldername using list of posts passed in
    
    Args:
        tmpl: name of templete
        foldername: name of folder to create
        posts: list of posts
    """
    
    template = lookup.get_template(tmpl)
    path = path_publish

    if foldername:
        path = path / foldername
    
    make_folder(path) # make folder in case it doesn't exist
    
    # get json for fusejs search object
    # todo: modify js to read json from disk instead of passing obj
    postsjson = search_json(posts, path)
    
    html = template.render(posts=posts, postsjson=postsjson).strip()
    
    # write to disk
    path = path / tmpl
    path.write_text(html)
    print(f"wrote {tmpl} to {path}")

def write_tags(posts=posts):
    """writes an index.html file for each tag"""
    for tag in tags.keys():
        # filter posts
        tag_posts = [post for post in posts if tag in post.tags]
        # using the same index page for now
        write_index_page(foldername=tag, posts=tag_posts)

write_index_page(foldername="", posts=posts)
write_tags()
write_posts(posts=posts)

wrote index.html to dist/index.html
wrote index.html to dist/python/index.html
wrote index.html to dist/jupyter/index.html
wrote index.html to dist/chromebooks/index.html
wrote index.html to dist/books/index.html
wrote index.html to dist/statistics/index.html
wrote index.html to dist/apps/index.html
wrote anaconda to dist/python/anaconda.html
wrote stats-done-wrong-book to dist/books/stats-done-wrong-book.html
wrote building_this_blog to dist/python/building_this_blog.html
wrote note_in_subfolder to dist/python/note_in_subfolder.html
wrote vs_code to dist/apps/vs_code.html
wrote jupyterlab to dist/jupyter/jupyterlab.html
wrote monty_hall to dist/python/monty_hall.html
wrote setup-crostini-chromebook to dist/chromebooks/setup-crostini-chromebook.html


In [25]:
post.tags[0], post.slug

('chromebooks', 'setup-crostini-chromebook')

In [26]:
# make the big picture templates
for tmpl in ["index.html"]:
    template = lookup.get_template(tmpl)
    html = template.render(posts=posts, postsjson=postsjson).strip()
    path = path_publish / tmpl
    path.write_text(html)
    print(f"wrote {tmpl} to {path}")

# write all the posts
template = lookup.get_template("post.html")
for post in posts:
    html = template.render(post=post).strip()
    path = path_publish / f"{post.slug}.html"
    path.write_text(html)
    print(f"wrote {post.slug} to {path}")

wrote index.html to dist/index.html
wrote stats-done-wrong-book to dist/stats-done-wrong-book.html
wrote building_this_blog to dist/building_this_blog.html
wrote anaconda to dist/anaconda.html
wrote note_in_subfolder to dist/note_in_subfolder.html
wrote vs_code to dist/vs_code.html
wrote jupyterlab to dist/jupyterlab.html
wrote setup-crostini-chromebook to dist/setup-crostini-chromebook.html


In [37]:
os.listdir(path_publish)

['anaconda.html',
 'building_this_blog.html',
 'vs_code.html',
 'stats-done-wrong-book.html',
 'setup-crostini-chromebook.html',
 'jupyterlab.html',
 'note_in_subfolder.html',
 'index.html']

In [41]:
for post in posts:
    print(post.tags)

['books', 'statistics']
['python']
['python']
['python']
['apps']
['jupyter']
['chromebooks']


In [63]:
def searchlist(posts):
    """returns a json string of a list of search stuff"""
    search_list = [{"title": post.title, "tags": post.tags, "link": f"{post.slug}.html"} for post in posts]
    return json.dumps(search_list)

postsjson = searchlist(posts)
postsjson

'[{"title": "Statistics Done Wrong: The Woefully Complete Guide", "tags": ["books", "statistics"], "link": "stats-done-wrong-book.html"}, {"title": "how the blog was built", "tags": ["python"], "link": "building_this_blog.html"}, {"title": "Using Anaconda", "tags": ["python"], "link": "anaconda.html"}, {"title": "Testing note in a subfolder", "tags": ["python"], "link": "note_in_subfolder.html"}, {"title": "Visual Studio Code", "tags": ["apps"], "link": "vs_code.html"}, {"title": "Jupyter Lab tips and tricks", "tags": ["jupyter"], "link": "jupyterlab.html"}, {"title": "Setting up Linux on a Chromebook with Crostini", "tags": ["chromebooks"], "link": "setup-crostini-chromebook.html"}]'

In [15]:
from nbconvert import HTMLExporter

In [13]:
path_nb = blog.path_nb

nb_paths = [f for f in path_nb.rglob("*.ipynb")]
nb_paths

[PosixPath('notebooks/monty_hall.ipynb')]

In [14]:
help(nbconvert.writers.FilesWriter)

Help on class FilesWriter in module nbconvert.writers.files:

class FilesWriter(nbconvert.writers.base.WriterBase)
 |  FilesWriter(*args, **kwargs)
 |  
 |  Consumes nbconvert output and produces files.
 |  
 |  Method resolution order:
 |      FilesWriter
 |      nbconvert.writers.base.WriterBase
 |      nbconvert.utils.base.NbConvertBase
 |      traitlets.config.configurable.LoggingConfigurable
 |      traitlets.config.configurable.Configurable
 |      traitlets.traitlets.HasTraits
 |      traitlets.traitlets.HasDescriptors
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, **kw)
 |      Constructor
 |  
 |  write(self, output, resources, notebook_name=None, **kw)
 |      Consume and write Jinja output to the file system.  Output directory
 |      is set via the 'build_directory' variable of this instance (a 
 |      configurable).
 |      
 |      See base for more...
 |  
 |  ----------------------------------------------------------------------
 |  Data

In [42]:
os.path.relpath("tmp/img")

'tmp/img'

In [47]:
writer = nbconvert.writers.FilesWriter()
writer.relpath = "_files"
writer.relpath

'_files'

In [77]:
os.path.relpath("tmp/kk")

'tmp/kk'

In [ ]:
# move notebook outputs to the right directory

In [95]:
def convert_notebooks_to_md():
    # emptying tmp so don't end up with old md files here
    shutil.rmtree("tmp", ignore_errors=True)

    exporter = nbconvert.MarkdownExporter()
    writer = nbconvert.writers.FilesWriter(
            build_directory="tmp",
            )

    for nb_path in nb_paths:
        print(f"converting {nb_path}")
        
        with open(nb_path) as f:
            nb_node = nbformat.read(f, as_version=4)
        #os.system(f'jupyter nbconvert --to markdown {nb_path} --output-dir tmp')
        #return None

        (body, resources) = exporter.from_notebook_node(nb_node)

        writer.write(
                    output=body,
                    resources=resources,
                    notebook_name=nb_path.name[:-6]
                )
        print(f"\nconverted {len(nb_paths)} notebooks to markdown files")
    return resources

    
r = convert_notebooks_to_md()

converting notebooks/monty_hall.ipynb

converted 1 notebooks to markdown files


In [87]:
print(r.keys())
r["global_content_filter"]

dict_keys(['metadata', 'output_extension', 'outputs', 'raw_mimetypes', 'global_content_filter'])


{'include_code': True,
 'include_markdown': True,
 'include_raw': True,
 'include_unknown': True,
 'include_input': True,
 'include_output': True,
 'include_input_prompt': True,
 'include_output_prompt': True,
 'no_prompt': False}

In [89]:
r["outputs"].keys()

dict_keys(['output_12_0.png', 'output_20_0.png'])

In [74]:
resources.keys()

dict_keys(['metadata', 'output_extension', 'outputs', 'raw_mimetypes', 'global_content_filter'])

In [48]:
resources["global_content_filter"]

{'include_code': True,
 'include_markdown': True,
 'include_raw': True,
 'include_unknown': True,
 'include_input': True,
 'include_output': True,
 'include_input_prompt': True,
 'include_output_prompt': True,
 'no_prompt': False}

In [18]:
[f for f in blog.path_nb.rglob("*.ipynb") if ".ipynb_checkpoints" not in f]

TypeError: argument of type 'WindowsPath' is not iterable

In [16]:
blog.path_nb

WindowsPath('notebooks')

In [22]:
blog.path_nb.

AttributeError: 'WindowsPath' object has no attribute 'path'

In [39]:
p = Path("notebooks")
ff = [f for f in p.rglob("*.ipynb") if ".ipynb_checkpoints" not in str(f)]
ff

[WindowsPath('notebooks/monty_hall.ipynb')]

In [38]:
f = ff[1]
str(f)

'notebooks\\.ipynb_checkpoints\\monty_hall-checkpoint.ipynb'

In [30]:
p / ".ipynb_checkpoints/" / "mon"

WindowsPath('notebooks/.ipynb_checkpoints')

In [43]:
p = Path("tmp")
pp = [p for p in os.listdir(p) if "_files" in p]
pp

['monty_hall_files']

In [52]:
 cc = [x for x in p.iterdir() if x.is_dir() and ("_files" in x.name)]
 cc

[WindowsPath('tmp/monty_hall_files')]

In [54]:
blog.path_publish

WindowsPath('dist')

In [60]:
for f in cc:
    print(f)
    shutil.copytree(src=f, dst=blog.path_publish / "python", dirs_exist_ok=True)

tmp\monty_hall_files


In [85]:
from nbdev import export2html
from nbdev import export
from nbdev.export2html import Config, Path, _re_digits, _to_html, _re_block_notes

In [94]:
e = export2html.MarkdownExporter()
e.from_file(fname='notebooks/monty_hall.ipynb')

TypeError: from_file() missing 1 required positional argument: 'file_stream'

In [84]:
export2html.notebook2html(fname='notebooks/*.ipynb', dest='tmp/', template_file='fastpages.tpl')

In [68]:
export2html.notebook2html

In [81]:
export2html.convert_md(fname='notebooks/monty_hall.ipynb', dest_path='tmp/', img_path=".", jekyll=True)

In [97]:
url = f"https://github.com/khalido/blog/blob/master/notebooks/{nb_path}"

NameError: name 'nb_path' is not defined